In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage

In [ ]:
# .env dosyası içerisinde gerekli olan keyler alınır
from dotenv import load_dotenv
_ = load_dotenv()


# Key bilgisi sağlamanın diğer yolu

# import openai
# openai.api_key = "YOUR_OPENAI_API_KEY"

In [ ]:
# LLM modelini tanımla
llm = OpenAI(model="gpt-4o-mini")

In [22]:
# Query Rewriting

def rewrite_query(query):
    prompt = f"Rewrite the following query to be more precise for search: {query}"

    messages = [
        ChatMessage(
            role="system", content=prompt
        ),
    ]
    
    response = llm.chat(messages)
    return response.message.content

In [ ]:
# 3. Load and Embed Documents
data_dir = "data"
reader = SimpleDirectoryReader(data_dir)  # 'data' klasöründeki belgeleri okur
docs = reader.load_data()

embedding_model = OpenAIEmbedding(model="text-embedding-ada-002",vector_store_query_mode='default')
# vector_store_query_mode:
# default: Standart benzerlik araması (kozünüs benzerliği kullanır).
# mmr: Maximal Marginal Relevance (MMR) ile çeşitliliği artırır. Aynı türden belgelerin tekrar etmesini engeller.
# hybrid: Vektör benzerliği + Anahtar kelime eşleşmesi kombinasyonu kullanır (TF-IDF veya BM25 ile).

index = VectorStoreIndex.from_documents(docs, embed_model=embedding_model)
retriever = index.as_retriever(similarity_top_k=7)

In [5]:
from sentence_transformers import SentenceTransformer

# Rerank için Bi-Encoder modelini yükle
bi_encoder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

c:\Users\Fatma Ceyhan\.conda\envs\rag\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
def search_documents(query):
    # Daha iyi bir arama için sorgunun rewrite edilmesi
    rewrited_query = rewrite_query(query)

    # Belge alma (Retriever ile)
    retrieved_docs = retriever.retrieve(rewrited_query)  # İlk K doküman

    # Eğer belge yoksa, boş döndür
    if not retrieved_docs:
        return []

    # Belge metinlerini ve vektörlerini al
    doc_texts = [doc.text for doc in retrieved_docs]
    doc_embeddings = bi_encoder.encode(doc_texts, normalize_embeddings=True)

    # Sorgu vektörünü hesapla
    query_embedding = bi_encoder.encode(query, normalize_embeddings=True)

    # Re-Rank aşaması (Dot Product kullanarak sıralama)
    rerank_scores = query_embedding @ doc_embeddings.T  # Dot product similarity

    # Dokümanları yeni skorlara göre sırala
    ranked_docs = sorted(zip(doc_texts, rerank_scores), key=lambda x: x[1], reverse=True)

    # En iyi 7 dokümanı döndür
    return [doc[0] for doc in ranked_docs[:4]], rewrited_query

In [24]:
def generate_response(query):
    relevant_docs, rewrited_query = search_documents(query)
    context = "\n".join(relevant_docs)
    prompt = f"Given the following documents, answer the query concisely.\n\nContext:\n{context}\n\nQuery: {rewrited_query}" 
    
    messages = [
        ChatMessage(
            role="system", content=prompt
        ),
    ]

    response = llm.chat(messages)
    return response.message.content, prompt

In [20]:
# Test
query = "What are the latest AI trends?"
#query = "What is AI?"
#query = "What is relationship between AI and Autonomous systems?"
#query = "What is AI history an relationship between AI and Autonomous systems?"
#query = "What is relationship among AI, Finance, Machine Learning and Autonomous systems?"
response, prompt = generate_response(query)
print(f"Prompt: \n{prompt}")
print(f"Answer: {response}")

Prompt: 
Given the following documents, answer the query concisely.

Context:
AI is rapidly evolving with trends like multimodal learning, smaller efficient models, and AI safety improvements.
The concept of AI dates back to ancient myths and early philosophical discussions, but it became a formal field of study in the 1950s. Early AI models focused on rule-based systems, while modern AI leverages deep learning and neural networks to achieve impressive levels of intelligence. From IBM's Deep Blue defeating a chess champion to OpenAI's ChatGPT generating human-like text, AI has come a long way.
AI continues to evolve rapidly, with advancements in general AI, explainable AI, and AI-human collaboration. Researchers are exploring artificial general intelligence (AGI), where machines could possess human-like cognitive abilities. The future of AI could reshape society, making tasks easier while raising new ethical and economic considerations.
Businesses leverage AI to optimize operations, de